In [35]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
from datetime import date
from itertools import cycle
from func_timeout import func_timeout, FunctionTimedOut
import pickle
import pandas_dedupe
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Matching names

### Reading all tables

In [36]:
with open("rev_table_processed.pkl", "rb") as fp:  
    mezcal_rev = pickle.load(fp)

In [37]:
mezcal_rev['origin'] = 'MR'

In [38]:
rename_cols = {}    
for col in mezcal_rev.columns:
    if col != 'origin' and col!='Bottle Name':
        rename_cols[col] = col+'_MR' 

In [39]:
mr_cols = ['Bottle Name', 'origin','Rating_MR', 'Cost_MR', 'Brand_MR', 'NOM_MR', 'Mezcalero_MR',
           'Agave_MR', 'Distillation_MR', 'Style_MR', 'State_MR',
           'Town_MR', 'ABV_MR', 'Website_MR', 'Image Link_MR', 'Review link_MR',]

In [40]:
mezcal_rev = mezcal_rev.rename(columns=rename_cols)[mr_cols]
mezcal_rev = mezcal_rev.rename(columns={'Cost_MR':'Price_MR','Distillation_MR':'Destillation_MR','Review link_MR':'Bottle_link_MR'})


mezcal_rev.head(1)

,Bottle Name,origin,Rating_MR,Price_MR,Brand_MR,NOM_MR,Mezcalero_MR,Agave_MR,Destillation_MR,Style_MR,State_MR,Town_MR,ABV_MR,Website_MR,Image Link_MR,Bottle_link_MR
0,Mitre Origen,MR,3,4.0,Mitre Mezcal,NOM-O338X,Don Tacho,Espadin,Copper,Joven,Oaxaca,Santiago Matatlan,37.5-40%,https://www.facebook.com/mezcalmitre/,https://www.mezcalreviews.com/wp-content/uploa...,https://www.mezcalreviews.com/mezcal/mitre-joven/


### old_tequila

In [41]:
with open("old_tequila_processed.pkl", "rb") as fp:  
       old_tequila = pickle.load(fp)

In [42]:
old_tequila = old_tequila.rename(columns={'bot_link':'Bottle Link_OT', 
                                          'bot_brand':'Brand_OT',
                                          'bot_price':'Price_OT',
                                          'bot_rating':'Rating_OT',
                                          'bot_name':'Bottle Name',
                                          'bot_num_reviews':'# Reviews'})
old_tequila['origin'] = 'OT'
old_tequila = old_tequila[['Bottle Name', 'origin', 'Price_OT', 'Brand_OT','Bottle Link_OT', 'Rating_OT','Image_OT']]
old_tequila.head(1)

,Bottle Name,origin,Price_OT,Brand_OT,Bottle Link_OT,Rating_OT,Image_OT
0,400 Conejos Espadin Mezcal Artesanal,OT,$27.99,400 Conejos,https://www.oldtowntequila.com/400-conejos-esp...,3,https://cdn11.bigcommerce.com/s-u9ww3di/images...


### mezcalia

In [43]:
with open("mezcalia_df_processed.pkl", "rb") as fp:  
    mezcalia = pickle.load(fp)

In [44]:
mezcalia = mezcalia.rename(columns={'website_link':'Bottle Link_MZ', 
                                    'bottle_price':'Price_MZ',
                                    'agave':'Agave_MZ',
                                    'ABV_etiq':'ABV_MZ',
                                    'bottle_name':'Bottle_Name',
                                    'destillation':'Destillation_MZ',
                                    #'Style':'Style_MZ',
                                    'aged':'Style_MZ',
                                    'agave':'Agave_MZ',
                                    'region':'State_MZ',
                                    'main_img':'Image_MZ',
                                    'bottle_name':'Bottle Name'
                                   })
mezcalia['origin'] = 'MZ'

In [45]:
MZ_cols = ['Bottle Name','origin','Price_MZ','ABV_MZ','Destillation_MZ',
             'Style_MZ','Agave_MZ','State_MZ','Image_MZ','Bottle Link_MZ']
mezcalia = mezcalia[MZ_cols]

mezcalia.head(1)

,Bottle Name,origin,Price_MZ,ABV_MZ,Destillation_MZ,Style_MZ,Agave_MZ,State_MZ,Image_MZ,Bottle Link_MZ
0,Mezcal Gin,MZ,750,40,destilado de metal,otros,cenizo,durango,https://mezcalia.com/wp-content/uploads/2017/0...,https://mezcalia.com/producto/mezcal-gin/


### Drizly

In [46]:
with open("drizly_data_raw.pkl", "rb") as fp:  
    drizly = pickle.load(fp)

In [47]:
drizly = drizly.rename(columns={'links':'Bottle Link_DZ', 
                                'price-one':'Price_DZ',
                                'price-one-bottle':'Bottle ml_DZ',
                                'price-two':'Price-two_DZ',
                                'price-two-bottle':'Bottle-two ml_DZ',
                                'Town':'Town_DZ',
                                'rateing':'Rating_DZ',
                                'ABV':'ABV_DZ',
                                'Town':'Town_DZ',
                                'name':'Bottle Name',
                                'image':'Image_DZ'})
drizly['origin'] = 'DRZL'

In [48]:
drzl_cols = []
for col in drizly.columns:
    if 'Bottle Name' in col or 'origin' in col or 'DZ' in col:
        drzl_cols.append(col)

In [49]:
DZ_cols = ['Bottle Name','origin','Bottle Link_DZ','Image_DZ','Price_DZ','Bottle ml_DZ','Price-two_DZ',
             'Bottle-two ml_DZ','ABV_DZ','Town_DZ','Rating_DZ']

In [50]:
drizly = drizly[DZ_cols]

### total_wine and missionliquor

In [51]:
with open("total_wine_no_rev.pkl", "rb") as fp:  
    total_wine = pickle.load(fp)
total_wine['origin'] = 'TW'

In [52]:
with open("missionliquor_no_rev.pkl", "rb") as fp:  
    missionliquor = pickle.load(fp)
missionliquor['origin'] = 'ML'

In [53]:
with open("laeuropea_no_rev.pkl", "rb") as fp:  
    laeuropea = pickle.load(fp)
cols = ['Bottle Name', 'origin', 'Price_LE', 'Region_LE', 
        'Bottle Link_LE','Image_LE', 'ABV_LE', 'Bottle_Volume_LE', 'Agave_LE']
laeuropea['origin'] = 'LE'
laeuropea = laeuropea[cols]

In [54]:
with open("bodega_no_rev.pkl", "rb") as fp:  
    bodega = pickle.load(fp)
bodega = bodega[['Bottle Link_BD', 'origin', 'Bottle Name', 'Price_BD', 'Image_BD', 'Bottle Volume_BD', 'ABV_BD']]

In [55]:
with open("mercadode_no_rev.pkl", "rb") as fp:  
    mercadode = pickle.load(fp)
mercadode = mercadode[['Bottle_Link_MD', 'origin', 'Bottle Name', 'Price_MD', 'Brand_MD','Image_MD', 'Bottle Volume_MD']]

## Matching  Fuzzy Names

#### processing names

In [56]:
for table in [mezcal_rev, old_tequila, drizly, mezcalia, total_wine, missionliquor, laeuropea, bodega, mercadode]:
    table['Bottle Name'] = table['Bottle Name'].str.lower().str.replace(r'(mezcal )','').str.replace(r'( mezcal)','')
    table['Bottle Name'] = table['Bottle Name'].str.replace(r' \d\d\dml | \d\d\d ml','').str.replace(r' \d\d\dml|\d\d\d ML','')
    table['Bottle Name'] = table['Bottle Name'].str.replace(r' \d\d\dML| \d\d\dmL','').str.replace(r' \(\d\d\dML\)| \(\d\d\d ML\)','')

    table['Bottle Name'] = table['Bottle Name'].str.replace(r'( +)',' ').str.replace(r'( $|^ )','')
    for idx in table.index:
        table['Bottle Name'].loc[idx] = ' '.join([word[0].upper() + word[1:]  for word in str(table['Bottle Name'].loc[idx]).lower().split(' ')])

#### merging

In [57]:
cols = ['Bottle Name','origin']
products = pd.concat([mezcal_rev[cols], 
                      old_tequila[cols], 
                      drizly[cols], 
                      mezcalia[cols], 
                      total_wine[cols], 
                      missionliquor[cols],
                      laeuropea[cols],
                      bodega[cols], 
                      mercadode[cols]
                     ], axis=0, ignore_index=True)
products.shape

(2834, 2)

#### deduplicating

In [58]:
test = products.copy()
test['Bottle Name'] = test['Bottle Name'].str.lower().str.replace(r'(mezcal )','').str.replace(r'( mezcal)','')
idx = test[test.duplicated(subset=['Bottle Name'], keep='last')].index.to_list()
products = products.drop(idx).reset_index(drop=True)
products.shape

(1909, 2)

In [59]:
products['Match_MR'] = np.nan
products['Match_OT'] = np.nan
products['Match_DRZL'] = np.nan
products['Match_MZ'] = np.nan
products['Match_ML'] = np.nan
products['Match_TW'] = np.nan
products['Match_LE'] = np.nan
products['Match_BD'] = np.nan
products['Match_MD'] = np.nan

In [60]:
%%time
for i in range(products.shape[0]): 
    for match_name in old_tequila['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if products['origin'].loc[i] != 'OT' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_OT'].loc[i] = match_name
            pass
    
    for match_name in drizly['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if products['origin'].loc[i] !='DRZL' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_DRZL'].loc[i] = match_name
            pass
    
    for match_name in mezcalia['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if products['origin'].loc[i] !='MZ' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_MZ'].loc[i] = match_name
            pass
    
    for match_name in mezcal_rev['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='MR' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_MR'].loc[i] = match_name
            pass
    
    for match_name in total_wine['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='TW' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_TW'].loc[i] = match_name
            pass
    
    for match_name in missionliquor['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='ML' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_ML'].loc[i] = match_name
            pass
    
    for match_name in laeuropea['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='LE' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_LE'].loc[i] = match_name
            pass
    
    for match_name in bodega['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='BD' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_BD'].loc[i] = match_name
            pass
    
    for match_name in mercadode['Bottle Name']:
        key_name = products['Bottle Name'].loc[i]
        if  products['origin'].loc[i] !='MD' and fuzz.token_set_ratio(key_name, match_name) == 100:
            products['Match_MD'].loc[i] = match_name
            pass

CPU times: user 16min 17s, sys: 7.09 s, total: 16min 24s
Wall time: 18min 2s


In [61]:
with open('products_raw.pkl','wb') as f:
    pickle.dump(products, f)

In [62]:
with open('products_raw.pkl','rb') as f:
    products = pickle.load(f)
products.shape

(1909, 11)

## Deduplicating 

#### Searching for duplicates in Bottle Names and Matches

In [63]:
products_dedup = products.copy()

In [64]:
MR_dupl = [name for name in products_dedup['Match_MR'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(MR_dupl[1:])]
products_dedup.shape

(1438, 11)

In [65]:
OT_dupl = [name for name in products_dedup['Match_OT'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(OT_dupl[1:])]
products_dedup.shape

(1285, 11)

In [66]:
DRZL_dupl = [name for name in products_dedup['Match_DRZL'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(DRZL_dupl[1:])]
products_dedup.shape

(1248, 11)

In [67]:
MZ_dupl = [name for name in products_dedup['Match_MZ'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(MZ_dupl[1:])]
products_dedup.shape

(1238, 11)

In [68]:
ML_dupl = [name for name in products_dedup['Match_ML'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(ML_dupl[1:])]
products_dedup.shape

(1213, 11)

In [69]:
LE_dupl = [name for name in products_dedup['Match_LE'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(LE_dupl[1:])]
products_dedup.shape

(1201, 11)

In [70]:
BD_dupl = [name for name in products_dedup['Match_BD'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(BD_dupl[1:])]
products_dedup.shape

(1199, 11)

In [71]:
MD_dupl = [name for name in products_dedup['Match_MD'].unique() if name in products_dedup['Bottle Name'].unique()]
products_dedup = products_dedup[~products_dedup['Bottle Name'].isin(MD_dupl[1:])]
products_dedup.shape

(1195, 11)

In [72]:
#products.sort_values('Bottle Name')['Bottle Name']

# Merging all tables

In [73]:
mezcalia.dropna(subset=['Bottle Name'],inplace=True)
products_dedup.dropna(subset=['Bottle Name'],inplace=True)
products_dedup.shape

(1195, 11)

In [74]:
all_tables = pd.concat([mezcal_rev, 
                        old_tequila, 
                        drizly, 
                        mezcalia,
                        total_wine, 
                        missionliquor,
                        laeuropea,
                        bodega, 
                        mercadode], axis=0, ignore_index=True)

In [75]:
all_tables.drop_duplicates(subset=['Bottle Name'], keep='first', inplace=True)
drizly.drop_duplicates(subset=['Bottle Name'], keep='first', inplace=True)
total_wine.drop_duplicates(subset=['Bottle Name'], keep='first', inplace=True)

In [76]:
main_merge = pd.merge(products_dedup, all_tables, on=['Bottle Name', 'origin'], how='left', )
main_merge.shape
rename_cols = {}
for col in main_merge.columns[8:]:
    rename_cols[col] = col+'_keep'
main_merge = main_merge.rename(columns={'Bottle Name':'Bottle Name_MAIN','origin':'Origin_MAIN'})
main_merge.shape

(1195, 73)

In [77]:
merge_MR = pd.merge(main_merge, 
                     mezcal_rev, 
                     left_on=['Match_MR'],
                     right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
                     how='left')
merge_MR.shape

(1195, 89)

In [78]:
#main_merge[main_merge['Origin_MAIN'] == 'LE'][['Price_LE_keep','Price_OT_keep']]

In [79]:
merge_OT = pd.merge(merge_MR, 
                   old_tequila, 
                   left_on=['Match_OT'], 
                   right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
                   how='left')
merge_OT.shape

(1196, 96)

In [80]:
merge_DRZL = pd.merge(merge_OT, 
             drizly, 
             left_on=['Match_DRZL'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_DRZL.shape

(1196, 107)

In [81]:
merge_MZ = pd.merge(merge_DRZL, 
             mezcalia, 
             left_on=['Match_MZ'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_MZ.shape

(1196, 117)

In [82]:
merge_TW = pd.merge(merge_MZ, 
             total_wine, 
             left_on=['Match_TW'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_TW.shape

(1196, 125)

In [83]:
merge_LE = pd.merge(merge_TW, 
                    laeuropea, 
                    left_on=['Match_LE'], 
                    right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
                    how='left')
merge_LE.shape

(1197, 134)

In [84]:
merge_BD = pd.merge(merge_LE, 
             bodega, 
             left_on=['Match_BD'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_BD.shape

(1198, 141)

In [85]:
merge_MD = pd.merge(merge_BD, 
             mercadode, 
             left_on=['Match_MD'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_MD.shape

(1200, 148)

In [86]:
merge_ML = pd.merge(merge_MD, 
             missionliquor, 
             left_on=['Match_ML'], 
             right_on=['Bottle Name'], suffixes=('_gone', '_keep'),
             how='left')
merge_ML.shape

(1200, 153)

In [87]:
# dropping Names and Origins
del_cols = []
for col in merge_ML.columns:
    if ('Bottle Name' in col or 'origin' in col) and 'MAIN' not in col:
        del_cols.append(col)  
merge_ML = merge_ML.drop(columns = del_cols)

In [88]:
%%time
cols_pairs = []
for left_col in merge_ML.columns[8:]:
    for right_col in merge_ML.columns[8:]:
        if  '_gone' in left_col and left_col[0:-5]==right_col[0:-5] and left_col!=right_col: 
            pair = (left_col, right_col)
            cols_pairs.append(pair)
drop_cols = []
rename_cols = {}
for col_pair in cols_pairs:
    merge_ML[col_pair[1]].fillna(merge_ML[col_pair[0]], inplace=True)
    drop_cols.append(col_pair[0])
    rename_cols[col_pair[1]] = col_pair[1][0:-5]

CPU times: user 54 ms, sys: 1.98 ms, total: 56 ms
Wall time: 57.3 ms


In [89]:
merge_ML = merge_ML.replace({'nan':np.nan}).replace({'nan':np.nan,', , ,':','}, regex=True)
merge_ML = merge_ML.drop(columns=drop_cols).rename(columns=rename_cols)
merge_ML.shape

(1200, 73)

#### checking if inintial merge with retailers is correct

In [90]:
check_cols = ['Bottle Name_MAIN','Match_MR', 'Match_OT','Match_DRZL', 'Match_MZ', 'Match_ML', 'Match_TW','Style_MR',
              'Style_MZ','Brand_OT','Brand_MR', 'Agave_MR','Agave_MZ','ABV_MZ','ABV_DZ','ABV_MR','Town_DZ','Town_MR',
              'State_MR','State_MZ',]
test_merge = merge_ML[~merge_ML['Agave_MZ'].isna()][check_cols]

In [91]:
for col in check_cols:
    test_merge[col] = test_merge[col].astype(str).str.lower().str.replace(r'(mezcal )','').str.replace(r'( mezcal)','')

In [92]:
#test_merge['Match_MR'][100], merge_ML['Match_MR'][100]

In [93]:
idx_odd = []
for i in test_merge[(~test_merge['Agave_MR'].isna()) & (~test_merge['Agave_MZ'].isna())].index:
    if test_merge['Agave_MR'].loc[i] != test_merge['Agave_MZ'].loc[i]:
        #print(test_merge['Agave_MR'].loc[i],'-', test_merge['Agave_MZ'].loc[i])
        idx_odd.append(i)

for i in test_merge[(~test_merge['Brand_MR'].isna()) & (~test_merge['Brand_OT'].isna())].index:
    if test_merge['Brand_MR'].loc[i] != test_merge['Brand_OT'].loc[i]:
        print(test_merge['Brand_MR'].loc[i],'-', test_merge['Brand_OT'].loc[i])
        idx_odd.append(i)    

rio revuelto - nan
gin mg - nan
quiquiriqui - nan
el yope - el yope 
nan - benesin
nan - alipus 
convite - nan
los cuerudos - nan
nan - el yope 
cuishes - nan
iba - nan
cuishes - nan
iba - nan
burrito fiestero - nan
burrito fiestero - nan
alipus - nan


#### Blending columns in one

In [94]:
cols = []
for col in merge_ML.columns[8:]:
    for neighbor in merge_ML.columns:
        if col[:-3] == neighbor[:-3] and col !=neighbor and col not in cols and neighbor not in cols:
            cols.append(col)
            cols.append(neighbor)

for col in merge_ML.columns[8:]:
    if col not in cols:
        cols.append(col)

In [95]:
drop_cols = []
rename_cols = {}
for col in ['ABV','Agave','Brand','Destillation','Style','Town','State']:
    pat = col+'\_[A-Z]{2}'
    sim_cols = re.findall(pat,','.join(cols))
    cols_del = sim_cols[1:]
    drop_cols.extend(cols_del)
    keep_col = sim_cols[0]
    rename_cols[keep_col] = col
    for fill_col in cols_del:
        merge_ML[keep_col].fillna(merge_ML[fill_col], inplace=True)
    
merge_ML.drop(columns=drop_cols,inplace=True)
merge_ML=merge_ML.rename(columns=rename_cols)

In [96]:
all_cols = merge_ML.columns[0:8].to_list()
all_cols.extend(sorted(merge_ML.columns[8:]))

In [97]:
merge_ML = merge_ML[all_cols]

In [98]:
merge_ML[(merge_ML['Price_DZ'].isna()) &
         (merge_ML['Price_OT'].isna()) &
         (merge_ML['Price_ML'].isna()) &
         (merge_ML['Price_TW'].isna()) &
         (merge_ML['Price_MZ'].isna())].shape

(426, 60)

# Postprocessing

In [99]:
df = merge_ML.copy()

In [100]:
df = df.rename(columns = {'Bottle Name_MAIN':'Bottle Name','Origin_MAIN':'Origin','Image Link_MR':'Image_MR',
                         'Mezcalero_MR':'Mezcalero','Website_MR':'Brand Website','NOM_MR':'NOM'})

In [101]:
df.drop(columns=['Taste_TW'], inplace=True)
df.dropna(subset=['Bottle Name'], inplace=True)
df = df[~df['Bottle Name'].astype(str).str.lower().str.contains(r'caddy|regal')]
df.shape

(1189, 59)

#### duplicates

In [102]:
df['Bottle Name'] = df['Bottle Name'].str.replace('ñ','n').str.replace('á','a').str.replace('í','i').str.replace('ó','o').str.replace('Á','A').str.replace('ã','a')
df.drop_duplicates(subset=['Bottle Name'], keep='first', inplace=True)
df.dropna(subset=['Bottle Name'], inplace=True)
df.shape

(1168, 59)

In [103]:
test = df.copy()
dedup_test = pandas_dedupe.dedupe_dataframe(test,['Bottle Name'])
idx = dedup_test[(dedup_test['confidence']>0.5) & (~dedup_test['Bottle Name'].fillna('').str.contains(r'(\d$)'))].index.to_list()
len(idx)

importing data ...
reading from dedupe_dataframe_learned_settings
clustering...
# duplicate sets 74


52

In [104]:
%%time
idx_del=[]
for idx_one in idx:
    for idx_two in idx:
        if fuzz.token_set_ratio(test['Bottle Name'].loc[idx_one], test['Bottle Name'].loc[idx_two]) > 90 and idx_one!=idx_two and idx_two not in idx_del and idx_one not in idx_del:
            idx_del.append(idx_two)

CPU times: user 585 ms, sys: 4.72 ms, total: 590 ms
Wall time: 593 ms


In [105]:
idx_keep = [x for x in idx if x not in idx_del]
len(idx_del)

24

In [106]:
#df.loc[idx_del]

In [107]:
df.drop(idx_del, inplace=True)
df.shape

(1144, 59)

In [108]:
df.shape

(1144, 59)

#### ABV

In [109]:
df['ABV'] = df['ABV'].str.extract(r'(\d.+(?=%))')
df['ABV'] = df['ABV'].str.extract(r'(\d+\.\d|\d.+(?=\-)|\d+)')
df['ABV'] = df['ABV'].str.replace('%','').astype(float)

In [110]:
df['ABV'] = df['ABV'].apply(lambda x: np.nan if x < 20 else x)

#### Ml in Name

In [111]:
df['Bottle Name'] = df['Bottle Name'].str.replace(r' \d\d\dml | \d\d\d ml','').str.replace(r' \d\d\dml|\d\d\d ML','')
df['Bottle Name'] = df['Bottle Name'].str.replace(r' \d\d\dML| \d\d\dmL','').str.replace(r' \(\d\d\dML\)| \(\d\d\d ML\)','')

#### Names UPPER to Lower

In [112]:
for col in ['Bottle Name', 'Brand','Agave', 'Style', 'Destillation', 'Mezcalero', 'State', 'Town']:
    df[col] = df[col].str.replace(r'( +)',' ').str.replace(r'( $|^ )','')
    for idx in df.index:
        df[col].loc[idx] = ' '.join([word[0].upper() + word[1:]  for word in str(df[col].loc[idx]).lower().split(' ')])

#### Rating

In [113]:
df['Rating_MR'] = df['Rating_MR'].astype(str).str.extract(r'(\d\.\d|\d)').astype(float).replace(0,np.nan)
df['Rating_OT'] = df['Rating_OT'].astype(str).str.extract(r'(\d\.\d|\d)').astype(float).replace(0,np.nan)
df['Rating_DZ'] = df['Rating_DZ'].astype(str).str.extract(r'(\d\.\d|\d)').astype(float).replace(0,np.nan)

In [114]:
df['AVG Rating'] = df[['Rating_OT','Rating_MR','Rating_DZ']].mean(axis=1).round(1)

#### Right cols sequence

In [115]:
from df2gspread import gspread2df as g2d
#temp = g2d.download(gfile=spreadsheet_key, wks_name = 'Blended Data', credentials=credentials, col_names=True, row_names=True)

#### Price

In [116]:
df['Price_TW'] = df['Price_TW'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
df['Price_ML'] = df['Price_ML'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
df['Price_DZ'] = df['Price_DZ'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
df['Price_OT'] = df['Price_OT'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

fill_MZ = df['Price_MZ'].astype(str).str.extract(r'(\d+)')
df['Price_MZ'] = df['Price_MZ'].astype(str).str.extract(r'((?<=-)\d+)').fillna(fill_MZ).astype(float)

In [117]:
df['AVG Price'] = df[['Price_TW','Price_ML','Price_DZ','Price_OT','Price_LE','Price_BD', 'Price_MD']].mean(axis=1).round(0)

In [118]:
df['AVG Price'].fillna(df['Price_MZ'], inplace=True)

In [119]:
df['AVG Price'].isna().sum()

299

#### Region

In [120]:
df['State'].fillna(df['Region_LE'], inplace=True)
df.drop(columns=['Region_LE'], inplace=True)

#### Links

In [121]:
df['Bottle Links'] = pd.Series([''])
link_cols = ['Bottle Link_DZ','Bottle Link_ML', 'Bottle Link_MZ', 'Bottle Link_OT','Bottle Link_TW', 
             'Bottle_link_MR', 'Bottle Link_LE', 'Bottle Link_LE', 'Bottle Link_BD', 'Bottle_Link_MD']
for col in link_cols:
    df['Bottle Links'] = df[col].fillna('') + ' ; ' + df['Bottle Links'].fillna('')
df['Bottle Links'] = df['Bottle Links'].str.replace(r'([ ; ]+)',' ; ').str.replace(r'( ; $)','').str.replace(r'(^ ; )','')

#### Images

In [122]:
df['Image Links'] = pd.Series([''])
link_cols = ['Image_MR', 'Image_OT','Image_DZ', 'Image_ML','Image_MZ', 'Image_TW','Image_LE','Image_MD','Image_BD']
for col in link_cols:
    df['Image Links'] = df[col].fillna('').astype(str) + ' ; ' + df['Image Links'].fillna('')
df['Image Links'] = df['Image Links'].str.replace(r'([ ; ]+)',' ; ').str.replace(r'( ; $)','').str.replace(r'(^ ; )','')

In [123]:
df = df.replace({'Nan':np.nan})
df = df.replace({'nan':np.nan})

# To GS

In [129]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-sheets-270906-86a657daa9f6.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1KBlFSr2JDLz13XLduYzLvhMJNqYbUiyjzffmzNjghnw'

In [131]:
right_cols = ['Bottle Name','Brand','AVG Price', 'AVG Rating', 'Agave', 'Style', 'ABV', 'Destillation', 'Mezcalero', 
              'State','Town', 'NOM', 'Brand Website', 'Bottle Links','Image Links','Origin', 
              'Match_MR', 'Match_OT', 'Match_DRZL','Match_MZ', 'Match_ML', 'Match_TW','Match_LE','Match_BD','Match_MD',
              'Price_DZ', 'Price_ML', 'Price_MZ','Price_OT', 'Price_TW', 'Price_LE', 'Price_MD', 'Price_BD',
              'Rating_DZ', 'Rating_MR', 'Rating_OT', 
              'Bottle Link_DZ', 'Bottle Link_ML', 'Bottle Link_MZ', 'Bottle Link_OT','Bottle Link_LE',
              'Bottle Link_TW', 'Bottle_link_MR','Bottle_Link_MD','Bottle Link_BD', 'Image_MR','Image_OT', 'Image_DZ', 'Image_ML',
              'Image_MZ', 'Image_TW', 'Image_LE', 'Image_MD', 'Image_BD']

In [125]:
df.head(1)

,Bottle Name,Origin,Match_MR,Match_OT,Match_DRZL,Match_MZ,Match_ML,Match_TW,ABV,Agave,Bottle Link_BD,Bottle Link_DZ,Bottle Link_LE,Bottle Link_ML,Bottle Link_MZ,Bottle Link_OT,Bottle Link_TW,Bottle Volume_BD,Bottle Volume_MD,Bottle ml_DZ,Bottle ml_TW,Bottle-two ml_DZ,Bottle_Link_MD,Bottle_Volume_LE,Bottle_link_MR,Brand,Destillation,Image_MR,Image_BD,Image_DZ,Image_LE,Image_MD,Image_ML,Image_MZ,Image_OT,Image_TW,Match_BD,Match_LE,Match_MD,Mezcalero,NOM,Price-two_DZ,Price_BD,Price_DZ,Price_LE,Price_MD,Price_ML,Price_MR,Price_MZ,Price_OT,Price_TW,Rating_DZ,Rating_MR,Rating_OT,State,Style,Town,Brand Website,AVG Rating,AVG Price,Bottle Links,Image Links
0,Cuish Tobala,MR,NaN,NaN,NaN,NaN,NaN,NaN,48.0,Tobala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.mezcalreviews.com/mezcal/cuish-tob...,Cuish Mezcales,Copper,https://www.mezcalreviews.com/wp-content/uploa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Berta Vasquez,NOM-O606X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oaxaca,Joven,San Baltazar Chichicapam,http://www.mezcalescuish.mx/,NaN,NaN,https://www.mezcalreviews.com/mezcal/cuish-tob...,https://www.mezcalreviews.com/wp-content/uploa...


In [132]:
to_sheets = df[right_cols].fillna('-').copy()
to_sheets.shape

(1144, 54)

In [133]:
wks_name = 'Blended Data'
d2g.upload(to_sheets,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=False)

<Worksheet 'Blended Data' id:1279664021>